In [1]:
import pandas as pd

## Register Status

In [2]:
register_status = {}

# Registradores inteiros (x0..x31)
for i in range(32):
    register_status[f"x{i}"] = {"x": i, "x_type": "int", "writer": None}

# Registradores de ponto flutuante (f0..f31)
for i in range(32):
    register_status[f"f{i}"] = {"f": i, "f_type": "float", "writer": None}

# register_status


## Parser das instruções

In [3]:
OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}

In [4]:
instruction_status =[]
with open("tests/ex.s", "r") as file:
    for i, line in enumerate(file):
        op = OPCODES[line.split()[0]]
        if (op == 0 or op == 1):
            rd = line.split()[1].replace(",", "")
            rs1 = line.split()[2].split("(")[1].replace(")", "")
            rs2 = None
            imm = line.split()[2].split("(")[0]
        else:
            rd = line.split()[1].replace(",", "")
            rs1 = line.split()[2].replace(",", "")
            rs2 = line.split()[3].replace(",", "")
            imm = None          
        instruction_status.append({ 
            "inst": line.replace("\n", ""),
            "opcode": op,
            "rd": rd,
            "rs1": rs1,
            "rs2": rs2,
            "imm": imm,
            "issue": None,
            "read_operands": None,
            "execution_complete": None,
            "write_result": None,
            "issue": None, 
            "read": None, 
            "exec": None, 
            "write": None

        })


## Parser das functionals units

In [5]:
fus_configs = {}
with open("tests/uf_config2.in", 'r') as f:
    linhas = f.readlines()
    for linha in linhas:
        partes = linha.strip().split()
        fus_configs[partes[0]] = {'qtd': partes[1], 'cycles': partes[2]}
fus_configs

{'int': {'qtd': '2', 'cycles': '1'}, 'div': {'qtd': '1', 'cycles': '10'}}

In [6]:
fus_status = {}
for uf_type, config in fus_configs.items():
    if uf_type == 'int':
        op = 0
    elif uf_type == 'mult':
        op = 1
    elif uf_type == 'add':
        op = 2
    elif uf_type == 'div':
        op = 3
    for c in range(int(config['qtd'])):
        fus_status[f'{uf_type}{c+1}'] = {
            'Opcode': op, 
            'Busy': False, 
            'Op': None, 
            'Fi': None,
            'Fj': None, 
            'Fk': None, 
            'Qj': None, 
            'Qk': None,
            'Rj': False, 
            'Rk': False, 
            'Cycles_left': int(config['cycles']), 
            'Cycles': int(config['cycles'])
        }
        

## Tabela Resultado

In [8]:
# stages = {
#     "Header": ["Instruction/Cycle", "Issue", "Read", "Execute", "Write"],
#     "Instruction/Cicle": [None]*len(inst_status),
#     "Issue": [None]*len(inst_status),
#     "Read": [None]*len(inst_status),
#     "Execute": [None]*len(inst_status),
#     "Write": [None]*len(inst_status)
# }

## ISSUE

Verificar se a unidade funcional requerida está livre -- sem hazards estruturais
    
    Functional unit status -- verificada aqui

Verificar se nenhuma outra instrução vai escrever no rd -- sem WAW hazards
    
    Register Status -- verificado aqui

In [9]:
pc = 0
instr = instruction_status[pc]
opcode_i = instr['opcode']
rs1_i = instr['rs1']
rs2_i = instr['rs2']
rd_i = instr['rd']
for fu_name, fu in fus_status.items():
    # verifica se a unidade funcional corresponde à instrução
    # se a unidade funcional não está vazia
    # se nenhuma outra FU n está escrevendo no rd
    if fu['Opcode'] != opcode_i or fu['Busy'] or register_status[rd_i]['writer'] is not None:
        continue
    fu['Busy'] = True
    fu['Op'] = opcode_i
    fu['Fi'] = rd_i
    fu['Fj'] = rs1_i
    fu['Fk'] = rs2_i
    fu['Qj'] = register_status[rs1_i]['writer'] if rs1_i is not None else None
    fu['Qk'] = register_status[rs2_i]['writer'] if rs2_i is not None else None
    fu['Rj'] = fu['Qj'] is None
    fu['Rk'] = fu['Qk'] is None
    register_status[rd_i]['writer'] = fu_name
    instruction_status[pc]['issue']=pc
    pc+=1
    break

In [10]:
def issue(pc, instr, fus_status, register_status, instruction_status):
    opcode_i = instr['opcode']
    rs1_i = instr['rs1']
    rs2_i = instr['rs2']
    rd_i = instr['rd']
    for fu_name, fu in fus_status.items():
        # verifica se a unidade funcional corresponde à instrução
        # se a unidade funcional não está vazia
        # se nenhuma outra FU n está escrevendo no rd
        if fu['Opcode'] != opcode_i or fu['Busy'] or register_status[rd_i]['writer'] is not None:
            continue
        fu['Busy'] = True
        fu['Op'] = opcode_i
        fu['Fi'] = rd_i
        fu['Fj'] = rs1_i
        fu['Fk'] = rs2_i
        fu['Qj'] = register_status[rs1_i]['writer'] if rs1_i is not None else None
        fu['Qk'] = register_status[rs2_i]['writer'] if rs2_i is not None else None
        fu['Rj'] = fu['Qj'] is None
        fu['Rk'] = fu['Qk'] is None
        register_status[rd_i]['writer'] = fu_name
        instruction_status[pc]['issue']=pc
        pc+=1
        break

## Read Operands

verifica no `register_status` se alguma unidade funcional vai escrever nos operandos de leitura `rs1` e `rs2`

In [11]:
ready_rs1 = (rs1_i is None) or (register_status[rs1_i]['writer'] is None)
ready_rs2 = (rs2_i is None) or (register_status[rs2_i]['writer'] is None)

if ready_rs1 and ready_rs2:
    instr['read_operands'] = pc
    fu['Rj'] = False
    fu['Rk'] = False


## Scoreboard

### Montando a instrução novamente

In [12]:
def any_fu_busy(fus):
    for fu_name, fu in fus.items():
        if fu['Busy']:
            return True
    return False

In [23]:
pc = 0
cycle = 0

In [21]:
len(instruction_status)

6

In [ ]:
#  or any_fu_busy(fus_status)
while pc < len(instruction_status):
    # read operands
    for inst in instruction_status:
        if inst['issue'] is not None and inst['read_operands'] is None:
            

        print(inst)
        break
    #     if inst[]
    # issue(pc, instr, fus_status, register_status, instruction_status)
    # cycle+=1
    pc+=1


{'inst': 'fld f1, 0(x1)', 'opcode': 0, 'rd': 'f1', 'rs1': 'x1', 'rs2': None, 'imm': '0', 'issue': 0, 'read_operands': 1, 'execution_complete': None, 'write_result': None, 'read': None, 'exec': None, 'write': None}
{'inst': 'fld f1, 0(x1)', 'opcode': 0, 'rd': 'f1', 'rs1': 'x1', 'rs2': None, 'imm': '0', 'issue': 0, 'read_operands': 1, 'execution_complete': None, 'write_result': None, 'read': None, 'exec': None, 'write': None}
{'inst': 'fld f1, 0(x1)', 'opcode': 0, 'rd': 'f1', 'rs1': 'x1', 'rs2': None, 'imm': '0', 'issue': 0, 'read_operands': 1, 'execution_complete': None, 'write_result': None, 'read': None, 'exec': None, 'write': None}
{'inst': 'fld f1, 0(x1)', 'opcode': 0, 'rd': 'f1', 'rs1': 'x1', 'rs2': None, 'imm': '0', 'issue': 0, 'read_operands': 1, 'execution_complete': None, 'write_result': None, 'read': None, 'exec': None, 'write': None}
{'inst': 'fld f1, 0(x1)', 'opcode': 0, 'rd': 'f1', 'rs1': 'x1', 'rs2': None, 'imm': '0', 'issue': 0, 'read_operands': 1, 'execution_complete': N

In [ ]:
for idx, i in enumerate(stages['Instruction/Cicle']):
    if i is not None:
        if i['opcode'] == 0:
            rd = f"f{i['rd']}" if i['rd_type'] == 'float' else f"x{i['rd']}"
            rs1 = f"f{i['rs1']}" if i['rs1_type'] == 'float' else f"x{i['rs1']}"
            imm = i.get('imm', 0)
            i_mont = f"fld {rd}, {imm}({rs1})"
            stages['Instruction/Cicle'][idx] = i_mont 


In [ ]:
df_stages = pd.DataFrame({
    "Instruction/Cycle": stages["Instruction/Cicle"],
    "Issue": stages["Issue"],
    "Read": stages["Read"],
    "Execute": stages["Execute"],
    "Write": stages["Write"]
})

# df_stages["Issue"] = df_stages["Issue"].astype("Int64")

# df_stages.style.hide(axis="index")
print(df_stages.to_string(index=False))